In [163]:
from pathlib import Path
import re
from functools import reduce
from collections import deque
from itertools import pairwise

In [225]:
data = Path('Day19.txt').read_text().splitlines()
workflow_part_split = data.index('')
workflows = data[0:workflow_part_split]
parts = data[workflow_part_split+1:]

workflows = {workflow[0:ind]: workflow[ind+1:-1].split(',') for workflow in workflows if (ind:=workflow.index('{'))}
parts = [re.match('{x=(?P<x>\d+),m=(?P<m>\d+),a=(?P<a>\d+),s=(?P<s>\d+)}', part) for part in parts]

# Part 1

In [226]:
workflow = workflows['in']
part = parts[0]

def apply_workflow(workflow, part):

    for rule in workflow:
        if (split:=rule.find(':')) > -1:
            condition, out = rule.split(':')
            
            part_attr, op, value = re.match('([xmas])([<>])(\d+)', condition).groups()

            if op == '>':
                if int(part[part_attr]) > int(value):
                    return out
            else:
                if int(part[part_attr]) < int(value):
                    return out
        else:
            return rule
        
def apply_workflows(workflows, start, part):
    
    while start not in 'AR':
        start = apply_workflow(workflows[start], part)

    return start

In [227]:
sum([reduce(lambda a,b: a+b, map(int, part.groups())) for part in parts if apply_workflows(workflows, 'in', part) == 'A'])

401674

# Part 2

In [228]:
to_visit = deque([['in', 0]])

current_path = deque()

successful_paths = set()


while to_visit:
    workflow_key, depth = to_visit.pop()

    if ':' in workflow_key:
        condition, workflow_key = workflow_key.split(':')
    else:
        condition = ''



    while len(current_path) > 2*depth:
        current_path.pop()

    if workflow_key == 'A':
        current_path.append(condition)
        current_path.append('A')
        successful_paths.add(','.join(current_path))
        continue
    elif workflow_key == 'R':
        continue

    current_path.append(condition)
    current_path.append(workflow_key)


    workflow = workflows[workflow_key]

    outcomes = [[rule, depth+1] for rule in workflow]

    for outcome in outcomes:
        to_visit.append(outcome)
    
    

In [229]:
def apply_condition(part, condition, negate = False):
    part_attr, op, value = re.match('([xmas])([<>])(\d+)', condition).groups()

    if negate:
        if op == '<':
            op = '>='
        elif op == '>':
            op = '<='

    value = int(value)

    if part[part_attr]:
        lower, upper = part[part_attr]
    else:
        return part

    if op == '>':
        if upper <= value:
            part[part_attr] = []
        
        elif lower-1 <= value < upper:
            part[part_attr] = [value+1, upper]
        
    elif op == '<':
        if lower < value <= upper + 1:
            part[part_attr] = [lower, value-1]
        
        elif lower >= value:
            part[part_attr] = []

    elif op == '>=':
        if upper < value:
            part[part_attr] = []
        
        elif lower <= value <= upper:
            part[part_attr] = [value, upper]

    elif op == '<=':
        if lower <= value <= upper:
            part[part_attr] = [lower, value]
        
        elif lower > value:
            part[part_attr] = []

    return part

In [230]:
total = 0

for path in successful_paths:
    part = {'x': [1, 4000], 'm': [1, 4000], 'a': [1, 4000], 's': [1, 4000]}

    path = path.split(',')

    for i in range(len(path)//2 - 1):
        workflow_key = path[2*i+1]
        target_condition = path[2*i+2]
        target_outcome = path[2*i+3]
    
        workflow = workflows[workflow_key]


        for rule in workflow:
            if ':' in rule:
                condition, outcome = rule.split(':')
                part = apply_condition(part, condition, (outcome != target_outcome or condition != target_condition))

            else:
                condition = ''
                outcome = rule

            if outcome == target_outcome and condition == target_condition:
                break

    total += reduce(lambda a, b: a*b, [v[1]-v[0]+1 if v else 0 for k,v in part.items()])

print(total)



134906204068564
